# About

This notebook adds canopy height features to the points sampled in the notebook `2_sample_pts_from_polygos.ipynb`. Due to data availability, canopy height data will only be added to points from the years 2016, 2018, and 200. The notebook assumes the csv files with the sampled points are located in the 'temp' folder. The canopy height rasters for Santa Barbara County were obtained from the California Forest Observatory (CFO) using the `1_download_CFO_canopy_height_raster.ipynb` notebook and are located in the 'SantabarbaraCounty_lidar' folder. 

While adding the canopy height features, this notebook creates four additional temporary rasters from the CFO canopy height layer *H*in a given year. These layers are avg_lidar, max_lidar, min_lidar, and min_max_diff. For each year, the avg_lidar layer is created by replacing the value of a pixel *p* in *H* by the average of the values of *H* in a 3x3 window centered at *p* (a convolution of the raster *H* with a 3x3 matrix with constant weights 1/9). The max_lidar is created by replacing the value of a pixel *p* in *H* with the maximum value of *H* in a 3x3 window centered at *p*. The min_lidar layer is created similarly, taking the minimum value over the window. Finally, the min_max_diff layer is the difference between the max_lidar and the min_lidar layers. All the functions to create these raster layers and sample their information are in `lidar_sampling_functions`. 



**NOTEBOOK VARIABLES:**

- `years` (int array): years of the points that will have lidar features added. Must be a subset of [2016, 2018, 2020]

- `aois` (array): the areas of interest of the points which will have lidar features added. Must be a subset of `['campus_lagoon','carpinteria','gaviota', 'point_conception']`.

- `delete_pts` (bool): whether to delete the input files with the original points or not.

To add canopy height features to all points from all aois and all years with available data, you need to set `years = [2016, 2018, 2020]` and `aois = ['campus_lagoon','carpinteria','gaviota','point_conception']`.

Notes: there are no points sampled from point_conception on 2016. The notebook automatically excludes this option. 


**OUTPUT:**
For each csv file of points from the specified year and aoi, the notebook augments the features in the original csv (see notebook `2_sample_pts_from_polygons.ipynb`) with the columns: canopy height, avg_lidar, max_lidar, min_lidar, and min_max_diff. The values for these columns are obtained using the points to sample the canopy height rasters.
Each data frame is saved as a csv file in the 'temp' folder (one csv per area of interest oi and year combination) with the name `aoi_pts_spectral_lidar_year.csv.`

In [1]:
import os
import pandas as pd

import rasterio
from rasterio.crs import CRS

import sample_rasters as sr

# Specify notebook variables

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

years = [2016, 2018, 2020]

aois = ['campus_lagoon','carpinteria','gaviota','point_conception']

delete_pts = True
    
# ***************************************************
# ***************************************************

# Add canopy height data from year `lidar_year` to points from all aois in year `year`

In [ ]:
# check the years are from the set where data is available
lidar_years = list(set(years) & set([2016,2018,2020]))

for year in lidar_years:
    # ------------------------------
    # Open canopy height raster and create auxiliary min, max, and avg rasters
    lidar_rast_r = rasterio.open(sr.path_to_lidar(year))
    rast_name = 'SB_canopy_height_'+str(year) # give a name to read raster
    
    sr.min_raster(rast_reader = lidar_rast_r, rast_name = rast_name, n=3)
    sr.max_raster(rast_reader = lidar_rast_r, rast_name = rast_name, n=3)
    sr.avg_raster(rast_reader = lidar_rast_r, rast_name = rast_name, n=3)
    
    # file paths to auxiliary canopy height rasters
    lidar_fps = []
    for tag in ['_maxs', '_mins', '_avgs']:
        lidar_fps.append(os.path.join(os.getcwd(),
                                     'temp',
                                     rast_name+tag+'.tif'))

    # ------------------------------
    # Add lidar data for each aoi
    for aoi in aois:
        if ('point_conception' != aoi) or (year != 2016):  #there's no data for Point Conception on 2016
            
            ## Get points information from csv
            pts_fp = sr.path_to_spectral_pts(aoi, year)
            crs = CRS.from_string(pd.read_csv(pts_fp).pts_crs[0])
            pts = sr.geodataframe_from_csv(pts_fp, 'x','y',crs)
            pts_col = pts.to_crs(lidar_rast_r.crs).geometry

            ## Sample canopy_height at point, and max, min and avg canopy height around point
            lidar_samples = sr.sample_raster_from_pts(pts_col, lidar_rast_r, ['lidar'])

            maxs_rast_r = rasterio.open(lidar_fps[0])
            max_samples = sr.sample_raster_from_pts(pts_col, maxs_rast_r, ['max_lidar'])

            mins_rast_r = rasterio.open(lidar_fps[1])
            min_samples = sr.sample_raster_from_pts(pts_col, mins_rast_r, ['min_lidar'])

            avg_rast_r = rasterio.open(lidar_fps[2])
            avg_samples = sr.sample_raster_from_pts(pts_col, avg_rast_r, ['avg_lidar'])

            ## Add all LIDAR data to pts dataframe
            pts = pd.concat([pts, lidar_samples, max_samples, min_samples, avg_samples], axis=1)
            pts['min_max_diff'] = pts.max_lidar - pts.min_lidar
            
            pts.drop(['geometry'],axis=1, inplace=True) # remove geometry column (already have lat,lon and CRS)
            
            ## Save points with added LIDAR data
            ptslidar_fp = os.path.join(os.getcwd(), 
                                       'temp', 
                                       aoi +'_pts_spectral_lidar_'+str(year)+'.csv')
            pts.to_csv(ptslidar_fp, index=False)

            ## Delete original csv files (points without LIDAR)
            if delete_pts == True:
                os.remove(pts_fp)

    # ------------------------------
    # Delete auxiliary LIDAR rasters created for this year
    for fp in lidar_fps:
        os.remove(fp)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interfac